In [ ]:
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from math import sqrt

In [ ]:
alphapose_json_dir = Path("outputs/alphapose/2/json")
baseline_json_dir = Path("outputs/simple_landmarks/2/json")
alphapose_overlay_dir = Path("outputs/alphapose/2/overlays")
frames_dir = Path("frames/frames2")

In [ ]:
def draw_keypoints(image, points, color, label_prefix=""):
    for i, (x, y) in enumerate(points):
        if x > 0 and y > 0:
            cv2.circle(image, (int(x), int(y)), 5, color, -1)
            cv2.putText(image, f"{label_prefix}{i+1}", (int(x)+5, int(y)-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1, cv2.LINE_AA)

In [38]:
def euclidean_distance(p1, p2):
    return sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

In [39]:
key_order = [
    "nose", "left_eye", "right_eye", "thumb",
    "pointer_finger", "middle_finger", "ring_finger", "pinky_finger"
]

In [ ]:
# Load and map AlphaPose detections
alphapose_by_image = {}
for idx in range(1, 7):
    json_path = alphapose_json_dir / f"{idx:02d}_result.json"
    if not json_path.exists():
        continue

    with open(json_path, "r") as f:
        results = json.load(f)

    for item in results:
        image_name = item["image_id"]
        if image_name not in alphapose_by_image or item["score"] > alphapose_by_image[image_name]["score"]:
            alphapose_by_image[image_name] = item

In [ ]:
all_frame_errors = []

for i in range(1, 7):
    frame_name = f"frame_{i:02d}.jpg"
    baseline_path = baseline_json_dir / f"frame_{i:02d}.json"

    if not baseline_path.exists():
        continue
    alpha_data = alphapose_by_image.get(frame_name)
    if not alpha_data:
        continue

    # Load data
    with open(baseline_path, "r") as f:
        baseline_data = json.load(f)
    baseline_points = [(baseline_data[k]["coordinates"]["x"], baseline_data[k]["coordinates"]["y"]) for k in key_order]

    keypoints = alpha_data["keypoints"]
    alpha_points = [(keypoints[j], keypoints[j+1]) for j in range(0, len(keypoints), 3)]

    # Load image
    img_path = alphapose_overlay_dir / frame_name
    if not img_path.exists():
        img_path = frames_dir / frame_name
    img = cv2.imread(str(img_path))
    if img is None:
        continue
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Draw both sets of points
    draw_keypoints(img, baseline_points, (0, 255, 0), "B")
    draw_keypoints(img, alpha_points, (255, 0, 0), "A")

    # Compute and draw distances
    errors = []
    for b, a in zip(baseline_points, alpha_points[:len(baseline_points)]):
        dist = euclidean_distance(b, a)
        errors.append(dist)
        cv2.line(img, (int(b[0]), int(b[1])), (int(a[0]), int(a[1])), (255, 255, 0), 1)

    mean_error = np.mean(errors)
    max_error = np.max(errors)
    all_frame_errors.append(mean_error)

    # Display results
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.title(f"Frame {i:02d}: Mean error={mean_error:.2f}px | Max error={max_error:.2f}px")
    plt.axis("off")
    plt.show()

    print(f"\nFrame {i:02d} Results")
    for k, dist in zip(key_order, errors):
        print(f"  {k:15s}: {dist:7.2f}px")
    print(f"  Mean error: {mean_error:.2f}px | Max error: {max_error:.2f}px")

print(f"Average mean error across frames: {np.mean(all_frame_errors):.2f}px")
print(f"Best (lowest) frame mean error: {np.min(all_frame_errors):.2f}px")
print(f"Worst (highest) frame mean error: {np.max(all_frame_errors):.2f}px")